In [ ]:
!pip install google-generativeai

In [ ]:
! pip3 install --upgrade --quiet --user google-cloud-aiplatform==1.88.0 --no-warn-script-location

In [ ]:
from vertexai.generative_models import FunctionDeclaration

add_function = FunctionDeclaration(
    name="add",
    description="Add two numbers together",
    parameters={
        "type": "object",
        "properties": {
            "a": {"type": "number", "description": "First number"},
            "b": {"type": "number", "description": "Second number"}
        },
        "required": ["a", "b"]
    }
)

multiply_function = FunctionDeclaration(
    name="multiply",
    description="Multiply two numbers together",
    parameters={
        "type": "object",
        "properties": {
            "a": {"type": "number", "description": "First number"},
            "b": {"type": "number", "description": "Second number"}
        },
        "required": ["a", "b"]
    }
)

In [ ]:
from vertexai.generative_models import Tool, GenerativeModel

math_tool = Tool(
    function_declarations=[add_function, multiply_function]
)

model = GenerativeModel(
    model_name="gemini-2.0-flash-001",
    tools=[math_tool],
    system_instruction=(
        "Fulfill the user's instructions, including telling jokes. "
        "If asked to add or multiply numbers, call the provided functions. "
        "You may call one function after the other if needed. "
        "Repeat the result to the user."
    )
)

chat = model.start_chat()

In [ ]:
def handle_response(response):
    # If there is a function call then invoke it
    # Otherwise print the response.
    if response.candidates[0].function_calls:
        function_call = response.candidates[0].function_calls[0]
    else:
        print(response.text)
        return

    if function_call.name == "add":
        a = function_call.args["a"]
        b = function_call.args["b"]
        result = add(a, b)
        # Send the result back to the chat session with the model
        new_response = chat.send_message(str(result))
        handle_response(new_response)

    elif function_call.name == "multiply":
        a = function_call.args["a"]
        b = function_call.args["b"]
        result = multiply(a, b)
        new_response = chat.send_message(str(result))
        handle_response(new_response)
    else:
        print(function_call)

In [ ]:
from vertexai.generative_models import Tool, GenerativeModel

math_tool = Tool(
    function_declarations=[add_function, multiply_function]
)

model = GenerativeModel(
    model_name="gemini-2.0-flash-001",
    tools=[math_tool],
    system_instruction=(
        "Fulfill the user's instructions, including telling jokes. "
        "If asked to add or multiply numbers, call the provided functions. "
        "You may call one function after the other if needed. "
        "Repeat the result to the user."
    )
)

chat = model.start_chat()

In [ ]:
response = chat.send_message("Tell me a joke?")
handle_response(response)

response = chat.send_message("I have 7 pizzas each with 16 slices. How many slices do I have?")
handle_response(response)

response = chat.send_message("Doug brought 3 pizzas. Andrew brought 4 pizzas. How many pizzas did they bring together?")
handle_response(response)

response = chat.send_message("Doug brought 3 pizzas. Andrew brought 4 pizzas. There are 16 slices per pizza. How many slices are there?")
handle_response(response)

response = chat.send_message("Doug brought 4 pizzas, but Andrew dropped 2 on the ground. How many pizzas are left?")
handle_response(response)